In [7]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

df_1_3 = pd.read_csv('results/6_augmentation.tsv',  sep='\t')

In [8]:
#Table 4 DICE WT
models = df_1_3['NN-Architecture (nn-Unet, UNETR)'].unique()
metric = 'Dice'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300
# to save space in the paper and not write 0.44 and write 44 instead (only for Dice and Surface Dice)
multiplier = 100
reindex_list = ['1_reg', '2a_interp', '3a_atlas', '4a_resamp', '4b_n4', '4d_susan','6_hist','5_ss_shared']
rename_list = ['1. Inter modality', '2. Resampling', '3. Atlas', '4. Resampling to spacing', '4.a Bias', '4.b Denoising', '4.c Histogram', '4.d Skull']


def get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier):
    
    # mean
    temp_series = (np.round(df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model ].groupby([ 'Preprocessing step'])[metric].mean(),3)*multiplier).round(0)
    temp_series = temp_series.reindex(reindex_list)
    temp_series.index = rename_list
    temp_series_mean = temp_series

    #std
    temp_series = (np.round(df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model].groupby([ 'Preprocessing step'])[metric].std(),3)*multiplier).round(0)

    temp_series = temp_series.reindex(reindex_list)
    temp_series.index = rename_list
    temp_series_std = temp_series

    temp_mean_std = pd.concat([temp_series_mean, temp_series_std], axis=1)
    temp_mean_std.columns = ['Mean', 'Std']
    temp_mean_std['Mean (STD)'] = temp_mean_std.apply(lambda x: str(x['Mean']) + ' (' + str(x['Std']) + ')', axis=1)
    return temp_mean_std['Mean (STD)'] 

print('DICE scores')
for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print(data, model, metric)
        print(get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier))
        print( )

DICE scores
=======================NEW MODEL=======================
gbm nn_unet Dice
1. Inter modality             nan (nan)
2. Resampling                 nan (nan)
3. Atlas                      nan (nan)
4. Resampling to spacing    84.0 (12.0)
4.a Bias                      nan (nan)
4.b Denoising                 nan (nan)
4.c Histogram                 nan (nan)
4.d Skull                   87.0 (10.0)
Name: Mean (STD), dtype: object

bgpd nn_unet Dice
1. Inter modality             nan (nan)
2. Resampling                 nan (nan)
3. Atlas                      nan (nan)
4. Resampling to spacing    73.0 (20.0)
4.a Bias                      nan (nan)
4.b Denoising                 nan (nan)
4.c Histogram                 nan (nan)
4.d Skull                   77.0 (14.0)
Name: Mean (STD), dtype: object



In [9]:
## How to check where to put rows on the statistical significance:

def dropna_intersection(data_1: pd.Series, data_2: pd.Series):
    data_1 = data_1.reset_index(drop=True)
    data_2 = data_2.reset_index(drop=True)

    data_1 = data_1.dropna()
    data_2 = data_2.dropna()

    data_1 = data_1[data_1.index.isin(data_2.index)]
    data_2 = data_2[data_2.index.isin(data_1.index)]
    
    return data_1, data_2

In [10]:
import scipy
from statsmodels.stats.weightstats import ttost_ind, ttost_paired

metric = 'Dice'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300

for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        preproc_1 = "4a_resamp"
        for key in reindex_list:
            preproc_2 = key

            if key != "4a_resamp":

                data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
                data_1, data_2 = dropna_intersection(data_1, data_2)

                ttost = ttost_paired(np.array(data_1), np.array(data_2), -data_2.std()/4, data_2.std()/4)[0]/7
                ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7
                                
                #assert isna
                assert data_1.isnull().sum() == 0
                assert data_2.isnull().sum() == 0

                # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                if (ttest_ < 0.05)&(ttost > 0.05):
                    print(data, model, metric)
                    print(preproc_2, (np.round(data_1.mean(),3)*100).round(0).astype(int), '(',
                      (np.round(data_1.std(),3)*100).round(0).astype(int), ')',
                      ' vs ',\
                      (np.round(data_2.mean(),3)*100).round(0).astype(int), '(' ,
                      (np.round(data_2.std(),3)*100).round(0).astype(int),')')
                    print( "Difference is significant, p-value ", np.round(ttost,3))
                    print( )

=======================NEW MODEL=======================
gbm nn_unet Dice
5_ss_shared 84 ( 12 )  vs  87 ( 10 )
Difference is significant, p-value  0.106

bgpd nn_unet Dice
5_ss_shared 73 ( 20 )  vs  77 ( 14 )
Difference is significant, p-value  0.102



In [11]:
metric = 'Dice'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300

for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        preproc_1 = "4a_resamp"
        for key in reindex_list:
            preproc_2 = key

            if key != "4a_resamp":

                data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
                data_1, data_2 = dropna_intersection(data_1, data_2)

                ttost = ttost_paired(np.array(data_1), np.array(data_2), -data_2.std()/4, data_2.std()/4)[0]/7
                ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7
                                
                #assert isna
                assert data_1.isnull().sum() == 0
                assert data_2.isnull().sum() == 0

                # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                if (ttest_ < 0.05)&(ttost > 0.05):
                    print(data, model, metric)
                    print(preproc_2, (np.round(data_1.mean(),3)*100).round(0).astype(int), '(',
                      (np.round(data_1.std(),3)*100).round(0).astype(int), ')',
                      ' vs ',\
                      (np.round(data_2.mean(),3)*100).round(0).astype(int), '(' ,
                      (np.round(data_2.std(),3)*100).round(0).astype(int),')')
                    print( "Difference is significant, p-value ", np.round(ttest_,3))
                    print( )

=======================NEW MODEL=======================
gbm nn_unet Dice
5_ss_shared 84 ( 12 )  vs  87 ( 10 )
Difference is significant, p-value  0.0

bgpd nn_unet Dice
5_ss_shared 73 ( 20 )  vs  77 ( 14 )
Difference is significant, p-value  0.0



In [12]:
metric = 'Surface Dice 1mm'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300

for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        preproc_1 = "4a_resamp"
        for key in reindex_list:
            preproc_2 = key

            if key != "4a_resamp":

                data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
                data_1, data_2 = dropna_intersection(data_1, data_2)
                
                #assert isna
                assert data_1.isnull().sum() == 0
                assert data_2.isnull().sum() == 0

                ttost = ttost_ind(data_1, data_2, -data_2.std()/4, data_2.std()/4)[0]/7
                ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

                # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                if (ttest_ < 0.05)&(ttost > 0.05):
                    print(data, model, metric)
                    print(preproc_2, (np.round(data_1.mean(),3)*100).round(0).astype(int), '(',
                      (np.round(data_1.std(),3)*100).round(0).astype(int), ')',
                      ' vs ',\
                      (np.round(data_2.mean(),3)*100).round(0).astype(int), '(' ,
                      (np.round(data_2.std(),3)*100).round(0).astype(int),')')
                    print( "Difference is significant, p-value ", np.round(ttest_,3))
                    print( )

=======================NEW MODEL=======================
gbm nn_unet Surface Dice 1mm
5_ss_shared 64 ( 21 )  vs  70 ( 20 )
Difference is significant, p-value  0.0

